# Ask ChatGPT to write a program using what we learned from Part 2

Write a python script to run in a Jupyter notebook that reads the materials.csv file with Brand, product name, product code, color and hand written notes, looks for and downloads the brand image, looks for and downloads an image of that material (using a combination of brand, product name, product code and color when available and not unknown) saving the material images in a directory called images with names that are a unique combination of brand and product name. Then prints an image of the fabric in the left hand column of a two column table and in the right hand column of that table print the brand image above on one line each the product name, product number, color, and any hand written notes. Output this table in a PDF document. Use a web scraping approach using Python BeautifulSoup libraries.

Print this to a pdf file.

Format this as a nice table with the image on the left the name of the fabric on the right and the fabric name linked to the source. Put up to 8 images on one page.

In [ ]:
!pip install requests beautifulsoup4 fpdf pillow pandas reportlab

Successfully installed fpdf-1.7.2 numpy-2.3.1 pandas-2.3.1 pillow-11.3.0 pytz-2025.2 tzdata-2025.2

In [ ]:
import os
import csv
import requests
from PIL import Image
from io import BytesIO
from bs4 import BeautifulSoup
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Image as RLImage, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
import urllib.parse
import pandas as pd

# Constants
DATA_FILE = 'data/materials.csv'  # Path to your CSV
IMAGE_DIR = 'data/images/part4/'
BRAND_IMAGE_DIR = os.path.join(IMAGE_DIR, 'brands')
FABRIC_IMAGE_DIR = os.path.join(IMAGE_DIR, 'fabrics')
OUTPUT_PDF = 'data/materials_catalog_part4.pdf'

# Create directories
os.makedirs(BRAND_IMAGE_DIR, exist_ok=True)
os.makedirs(FABRIC_IMAGE_DIR, exist_ok=True)

# Helper Functions
def fetch_image_url(query):
    headers = {"User-Agent": "Mozilla/5.0"}
    search_url = f"https://www.google.com/search?tbm=isch&q={urllib.parse.quote(query)}"
    try:
        response = requests.get(search_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        images = soup.find_all("img")
        for img in images:
            src = img.get("src")
            if src and src.startswith("http"):
                return src
    except Exception as e:
        print(f"Failed to fetch image for {query}: {e}")
    return None

def download_image(url, save_path):
    try:
        response = requests.get(url, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image.save(save_path)
        return save_path
    except Exception as e:
        print(f"Failed to download image from {url}: {e}")
        return None

def create_pdf_from_data(df):
    doc = SimpleDocTemplate(OUTPUT_PDF, pagesize=letter)
    elements = []
    styles = getSampleStyleSheet()
    data = []
    
    for idx, row in df.iterrows():
        brand = row["Brand"]
        product_name = row["Product Name"]
        product_code = row["Product Code"]
        color = row["Color"]
        notes = row["Notes"]

        # Fabric image
        fabric_img_path = os.path.join(FABRIC_IMAGE_DIR, f"{brand}_{product_name}.jpg".replace(" ", "_"))
        if os.path.exists(fabric_img_path):
            fabric_img = RLImage(fabric_img_path, width=100, height=100)
        else:
            fabric_img = Paragraph("No Image", styles["Normal"])

        # Brand image
        brand_img_path = os.path.join(BRAND_IMAGE_DIR, f"{brand}.jpg".replace(" ", "_"))
        if os.path.exists(brand_img_path):
            brand_img = RLImage(brand_img_path, width=50, height=30)
        else:
            brand_img = Paragraph("No Logo", styles["Normal"])

        # Info paragraph
        info = [
            brand_img,
            Paragraph(f"<b>{product_name}</b>", styles["Normal"]),
            Paragraph(f"Code: {product_code}", styles["Normal"]),
            Paragraph(f"Color: {color}", styles["Normal"]),
            Paragraph(f"Notes: {notes}", styles["Normal"]),
        ]
        row_data = [fabric_img, info]
        data.append(row_data)

        # Add 8 items per page
        if (idx + 1) % 8 == 0:
            t = Table(data, colWidths=[110, 400])
            t.setStyle(TableStyle([
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                ('BOX', (0, 0), (-1, -1), 1, colors.black),
                ('INNERGRID', (0, 0), (-1, -1), 0.5, colors.grey),
            ]))
            elements.append(t)
            elements.append(Spacer(1, 12))
            data = []

    # Add remaining rows
    if data:
        t = Table(data, colWidths=[110, 400])
        t.setStyle(TableStyle([
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('BOX', (0, 0), (-1, -1), 1, colors.black),
            ('INNERGRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ]))
        elements.append(t)

    doc.build(elements)
    print(f"PDF saved to: {OUTPUT_PDF}")

# Main Script
df = pd.read_csv(DATA_FILE)

for _, row in df.iterrows():
    brand = row["Brand"]
    product_name = row["Product Name"]
    product_code = row["Product Code"]
    color = row["Color"]

    # Download brand logo
    brand_img_path = os.path.join(BRAND_IMAGE_DIR, f"{brand}.jpg".replace(" ", "_"))
    if not os.path.exists(brand_img_path):
        brand_url = fetch_image_url(f"{brand} logo")
        if brand_url:
            download_image(brand_url, brand_img_path)

    # Download fabric image
    fabric_img_path = os.path.join(FABRIC_IMAGE_DIR, f"{brand}_{product_name}.jpg".replace(" ", "_"))
    if not os.path.exists(fabric_img_path):
        query_parts = [brand, product_name]
        if product_code.lower() != "unknown":
            query_parts.append(product_code)
        if color.lower() != "unknown":
            query_parts.append(color)
        fabric_query = " ".join(query_parts) + " fabric"
        fabric_url = fetch_image_url(fabric_query)
        if fabric_url:
            download_image(fabric_url, fabric_img_path)

# Create the PDF
create_pdf_from_data(df)

# You will need a PDF reader
* Search and install the KDAN PDF Reader - View and Edit PDF
* Or download and install the free Adobe Acrobat Reader

In [ ]:
import sys
print(sys.executable)

In [ ]:
!{sys.executable} -m pip install reportlab